In [1]:
from __future__ import division
from full_prod_DRA import *
from buchi import buchi_from_ltl
import numpy as np
from env_sensing_error import *
import scipy
# from plot_path_for_prod import *
from graphviz import Source
from qnetwork import *
from utils import *
import matplotlib.pyplot as plt
# from Plot_Path import *
import tensorflow as tf
import sys
from dra_planning import *
import time

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def train(sess, env, qnet, prod_planner):
    
    global EXPLORATION_RATE
    global GUIDE_RATE
  
    summary_ops, summary_vars = build_summaries()
    if not RESTORE:
        sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    
    qnet.update_target()
    
    replay_buffer = ReplayBuffer(BUFFER_SIZE, RANDOM_SEED)
    
    while len(prod_planner.opt_path) == 0:
        env.step(np.random.randint(0,qnet.action_dim))
        prod_planner.update_wfts_ap()
        prod_planner.get_global_opt()
    print "Global Solution Found"
    prod_planner.get_opt_rabin()
    
    saved_dra_planners = {}
    
    for num_epi in range(MAX_EPISODES):
        
#         print "Epi: ", num_epi

        s = env.reset()
        s = [list(np.unravel_index(s, env.shape))]
        prod_planner.replace_region_list()

        ep_reward = 0
        ep_ave_max_q = 0
        
        reward_list = []
        
        train_time = 0
        batch_time = 0
        gym_time = 0
        guide_time = 0

        for j in range(MAX_EPISODE_LEN):
            
#             print "Step: ", j
            
            gym_start = time.time()

            rand_num = np.random.rand(1)
    
            if rand_num <= EXPLORATION_RATE:
                a = np.random.randint(0,qnet.action_dim)
                s2, r, terminal, info = env.step(a)
            elif EXPLORATION_RATE < rand_num <= GUIDE_RATE+EXPLORATION_RATE:
#                 print "GUIDE"
                guide_start = time.time()
                
                if rand_num > EXPLORATION_RATE + 0.9*GUIDE_RATE:
                # Only update global plan with 0.2 prob for efficiency
                    prod_planner.get_global_opt()
                    prod_planner.get_opt_rabin()
                    
                if len(prod_planner.opt_path) > 0:
                    new_ltl = prod_planner.get_next_ltl(s[-1])
                    
#                     if new_ltl in saved_dra_planners.keys():
                    guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
                    if guide_path != None:
                        print "GUIDE"
                        a = convert_path_to_action(guide_path)
                    else:
                        a = np.random.randint(0,qnet.action_dim)
#                     else:
#                         guide_path = prod_planner.get_local_opt(s[:-1], new_ltl)
#                         saved_dra_planners[new_ltl] = prod_planner.dra_full_prod
#                         if guide_path != None:
#                             a = convert_path_to_action(guide_path)
#                         else:
#                             a = np.random.randint(0,qnet.action_dim)
                        
                else:
                    a = np.random.randint(0,qnet.action_dim)
                s2, r, terminal, info = env.step(a)
                
                guide_time += time.time() - guide_start
                
            else:
                a = np.argmax(qnet.predict_q(np.reshape(s, (1, qnet.state_dim))))
                s2, r, terminal, info = env.step(a)

            prod_planner.update_wfts_ap()
                
            gym_time += time.time() - gym_start
            
            batch_start = time.time()
            
            s2 = list(np.unravel_index(s2, env.shape))

            replay_buffer.add(np.reshape(s, (qnet.state_dim,)), np.reshape(a, (1,)), r,
                              terminal, np.reshape(s2, (qnet.state_dim,)))
            batch_time += time.time() - batch_start

            # Keep adding experience to the memory until
            # there are at least minibatch size samples
            if replay_buffer.size() > MINIBATCH_SIZE:
                
                batch_start = time.time()
                s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(MINIBATCH_SIZE)

                # Calculate targets
                target_q = qnet.predect_target(s2_batch)

                y_i = []
                for k in range(MINIBATCH_SIZE):
                    if t_batch[k]:
                        y_i.append(r_batch[k])
                    else:
                        y_i.append(r_batch[k] + GAMMA * np.amax(target_q[k]))
                        
                batch_time += time.time() - batch_start

                # Update the critic given the targets
                train_start = time.time()
                predicted_q_value, _ = qnet.train(s_batch, a_batch, np.reshape(y_i, (MINIBATCH_SIZE, 1)), num_epi)

                ep_ave_max_q += np.amax(predicted_q_value)
                
                # Update target networks
                qnet.update_target()

                train_time += time.time() - train_start

            s = s2
            ep_reward += r

            if terminal or j == MAX_EPISODE_LEN-1:
                
                if EXPLORATION_RATE > 0.02 and terminal:
                    EXPLORATION_RATE = EXPLORATION_RATE*0.98
                    GUIDE_RATE = GUIDE_RATE*0.98
                    
                reward_list += [ep_reward]
                
                if np.average(reward_list[-10:]) > LR_DECAY_TRUNCATION:
                    qnet.decay_learning_rate(0.98)

                print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f} | Exploration: {:.6f} | Step: {:d} | LR: {:.8f}'.format(int(ep_reward), \
                        num_epi, (ep_ave_max_q / float(j)), EXPLORATION_RATE, j, qnet.get_learning_rate()))
                
                f = open("stats/" + file_appendix + "_stats.txt", "ab")
                f.write("| Reward: " + str(int(ep_reward)) 
                        +" | Episode: " + str(num_epi) 
                        + " | Qmax: " + str(ep_ave_max_q / float(j)) 
                        + " | Exploration: " + str(EXPLORATION_RATE)
                        + " | Step: " + str(j)
                        + " | LR:" + str(qnet.get_learning_rate()) + "\n")
                f.close()
                
                f = open("stats/" + file_appendix + "_stats_time.txt", "ab")
                f.write(" | Episode: " + str(num_epi) 
                        + " | Train: " + str(train_time) 
                        + " | Gym: " + str(gym_time)
                        + " | Batch: " + str(batch_time) 
                        + " | Guide: " + str(guide_time)
                        + "\n")
                f.close()
                
                break
                

In [3]:
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B"
# LTL = "[] (p1 -> !(X p1) U (p2 || p3) ) && []<>p1"
# LTL = "T && []<>A && []<>B"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C && []!D"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C"
# LTL = "<>(A && <>(B && <> T))"
# LTL = "<>(A && <>B) && <>[]T && []!C"
LTL = "<>(A && <>T) && []!C"
# LTL = "<>(A && <>(B && <>T)) && []<>(A||T) && []<>B && []!C"
# LTL = "<>(A && <>(B && <>T)) && []!C"
# LTL = "<>(A && <>D) && <>(B && <>E) && []<>T && []<>(D || E) && []!C"

LEARNING_RATE = 0.00001
GAMMA = 0.99
# GAMMA = 0.7
TAU = 0.001
BUFFER_SIZE = 10**6
MINIBATCH_SIZE = 1
RANDOM_SEED = 210
MAX_EPISODES = 50000
MAX_EPISODE_LEN = 500
file_appendix = "Guide_Planning_" + time.ctime()[4:16].replace("  ","").replace(" ","_").replace(":","-") + "_large_" + LTL
# file_appendix = "Feb8_03-47"
SUMMARY_DIR = './results/tf_ddqn_' + file_appendix
SAVE_DIR = "./saved_model/" + file_appendix + "/ddqn.ckpt"
EXPLORATION_RATE = 0.2
GUIDE_RATE = 0.6
LR_DECAY_TRUNCATION = -200
RESTORE = 0
if sys.platform == "darwin":
    DEVICE = "/device:CPU:0"
else:
    DEVICE = "/device:GPU:0"

In [4]:
env = CurrentWorld(LTL.lower())
with open("my.dot", "r") as dotfile:
    text = dotfile.read()
Source(text)

prod_planner = Prod_Planning(env, LTL.lower())

In [5]:
config=tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
       
    state_dim = 3
    action_dim = 5
    
    if RESTORE:
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)
        Qnet.saver.restore(sess, RESTORE_PATH)
        train(sess, env, Qnet, prod_planner)
        
    else:
        np.random.seed(RANDOM_SEED)
        tf.set_random_seed(RANDOM_SEED)
        env.seed(RANDOM_SEED)
    
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)

        train(sess, env, Qnet, prod_planner)

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Global Solution Found
DDQN Saved


KeyboardInterrupt: 

In [8]:
prod_planner.wfts.L(prod_planner.region_list[np.ravel_multi_index((5,5),(10,10))])

['r55', 'c']

In [11]:
old_planner = deepcopy(prod_planner)

In [71]:
env.ap_dict

{'A': [(6, 6), (8, 8)],
 'B': [(5, 2)],
 'C': [(2, 2), (3, 3), (4, 4), (5, 5), (2, 7)],
 'T': []}

In [50]:
env.dynamic_coord_dict[(8,8)]

['A']

In [70]:
env.last_ap_dict

{'A': [(2, 7), (8, 8)],
 'B': [(5, 2)],
 'C': [(2, 2), (3, 3), (4, 4), (5, 5), (6, 6)],
 'T': []}

In [16]:
prod_planner.update_wfts_ap()

In [17]:
prod_planner.get_global_opt()

In [51]:
old_planner.full_prod.transition[(old_planner.region_list[np.ravel_multi_index((2,6),(10,10))],1)]

KeyError: (r26, 1)

In [52]:
old_planner.full_prod.states

{(r94, 0),
 (r94, 1),
 (r94, 2),
 (r94, 3),
 (r94, 4),
 (r73, 0),
 (r73, 1),
 (r73, 2),
 (r73, 3),
 (r73, 4),
 (r85, 0),
 (r85, 1),
 (r85, 2),
 (r85, 3),
 (r85, 4),
 (r84, 0),
 (r84, 1),
 (r84, 2),
 (r84, 3),
 (r84, 4),
 (r27, 0),
 (r27, 1),
 (r27, 2),
 (r27, 3),
 (r27, 4),
 (r13, 0),
 (r13, 1),
 (r13, 2),
 (r13, 3),
 (r13, 4),
 (r26, 0),
 (r26, 1),
 (r26, 2),
 (r26, 3),
 (r26, 4),
 (r14, 0),
 (r14, 1),
 (r14, 2),
 (r14, 3),
 (r14, 4),
 (r28, 0),
 (r28, 1),
 (r28, 2),
 (r28, 3),
 (r28, 4),
 (r21, 0),
 (r21, 1),
 (r21, 2),
 (r21, 3),
 (r21, 4),
 (r25, 0),
 (r25, 1),
 (r25, 2),
 (r25, 3),
 (r25, 4),
 (r22, 0),
 (r22, 1),
 (r22, 2),
 (r22, 3),
 (r22, 4),
 (r76, 0),
 (r76, 1),
 (r76, 2),
 (r76, 3),
 (r76, 4),
 (r24, 0),
 (r24, 1),
 (r24, 2),
 (r24, 3),
 (r24, 4),
 (r74, 0),
 (r74, 1),
 (r74, 2),
 (r74, 3),
 (r74, 4),
 (r15, 0),
 (r15, 1),
 (r15, 2),
 (r15, 3),
 (r15, 4),
 (r95, 0),
 (r95, 1),
 (r95, 2),
 (r95, 3),
 (r95, 4),
 (r23, 0),
 (r23, 1),
 (r23, 2),
 (r23, 3),
 (r23, 4),
 (r10, 0),

In [73]:
sorted(list(old_planner.wfts.states), key=lambda x: int(x.__repr__()[1:]))[27].ap

['a', 'r27']

In [89]:
wfts_state = deepcopy(old_planner.wfts.states)

In [92]:
wfts_state.update([prod_planner.region_list[27]])

In [74]:
prod_planner.region_list[27]

['r27', 'c']

In [85]:
a = set([1,2,3])

In [88]:
a.update()

TypeError: 'int' object is not iterable

In [87]:
a

{1, 2, 3}

In [95]:
old_planner.wfts.transition[list(old_planner.wfts.states)[0]]

{r94: 1, r84: 1, r95: 1, r93: 1}

In [136]:
class region0(object):
    def __init__(self):
        self.ap = 1

In [148]:
class wfts0(object):
    def __init__(self, r_l):
        self.r_l = r_l
        self.transition = {}
        for i in r_l:
            if i not in self.transition.keys():
                self.transition[i] = {}
            self.transition[i][i] = 1
    def add_s(self, new):
        self.r_l += [new]

In [138]:
class Test0(object):
    def __init__(self):
        region_list = [region0() for i in range(10)]
        self.wfts = wfts0(region_list)
        self.region_list = region_list

In [141]:
a = Test0()

In [147]:
a.wfts.transition[a.region_list[0]]

{<__main__.region0 at 0x12d364bd0>: 1}

In [152]:
a = [[1,2,3]]

In [115]:
a.region_list[0]=10

In [144]:
a.region_list[0].ap =100

In [118]:
a.wfts.transition

{0: {1: 1},
 1: {2: 1},
 2: {3: 1},
 3: {4: 1},
 4: {5: 1},
 5: {6: 1},
 6: {7: 1},
 7: {8: 1},
 8: {9: 1},
 9: {10: 1}}

In [122]:
a = range(10)

In [123]:
a.remove(a[0])

In [124]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9]